In [55]:
import pandas as pd
import cv2
import glob
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [56]:
import os, sys, codecs, glob
from PIL import Image, ImageDraw

import numpy as np
import pandas as pd
import cv2

import torch
torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.enabled = False

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
import timm

In [57]:
class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_group, transform):
        self.img_path = img_path
        self.transform = transform
        self.group = img_group

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, self.group[index]

    def __len__(self):
        return len(self.img_path)

In [64]:
class ArcModule(nn.Module):
    def __init__(self, in_features, out_features, s = 10, m = 0.2):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_normal_(self.weight)

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = torch.tensor(math.cos(math.pi - m))
        self.mm = torch.tensor(math.sin(math.pi - m) * m)

    def forward(self, inputs, labels):
        cos_th = F.linear(inputs, F.normalize(self.weight))
        cos_th = cos_th.clamp(-1, 1)
        sin_th = torch.sqrt(1.0 - torch.pow(cos_th, 2))
        cos_th_m = cos_th * self.cos_m - sin_th * self.sin_m
        # print(type(cos_th), type(self.th), type(cos_th_m), type(self.mm))
        cos_th_m = torch.where(cos_th > self.th, cos_th_m, cos_th - self.mm)
        
        cond_v = cos_th - self.th
        cond = cond_v <= 0
        cos_th_m[cond] = (cos_th - self.mm)[cond]

        if labels.dim() == 1:
            labels = labels.unsqueeze(-1)
        onehot = torch.zeros(cos_th.size()).cuda()
        labels = labels.type(torch.LongTensor).cuda()
        onehot.scatter_(1, labels, 1.0)
        outputs = onehot * cos_th_m + (1.0 - onehot) * cos_th
        outputs = outputs * self.s
        return outputs
    
    
class XunFeiNet_a(nn.Module):
    def __init__(self):
        super(XunFeiNet_a, self).__init__()
                
        model = timm.create_model('efficientnet_b3', pretrained=True)
        self.model = model
        model.classifier = torch.nn.Identity()
        self.margin = ArcModule(in_features=1536, out_features = 1806)
        
    def forward(self, img, labels=None):        
        feat = self.model(img)
        
        feat = F.normalize(feat)
        if labels is not None:
            return self.margin(feat, labels)
        return feat



class XunFeiNet_b(nn.Module):
    def __init__(self):
        super(XunFeiNet_b, self).__init__()
                
        model = timm.create_model('efficientnet_b3', pretrained=True)
        self.model = model
        model.classifier = torch.nn.Identity()
        self.margin = ArcModule(in_features=1536, out_features = 1807)
        
    def forward(self, img, labels=None):        
        feat = self.model(img)
        
        feat = F.normalize(feat)
        if labels is not None:
            return self.margin(feat, labels)
        return feat
    
model_a = XunFeiNet_a().cuda()
model_b = XunFeiNet_b().cuda()

In [65]:
test_path = glob.glob('data/test_no_face/*')
test_path.sort()
test_path = np.array(test_path)

test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path, [0]*len(test_path),
                        transforms.Compose([
                        transforms.Resize((320, 320)),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ),
    batch_size=50, shuffle=False, num_workers=5,
)

In [97]:
# 加载模型
from sklearn.preprocessing import normalize
model_a.load_state_dict(torch.load('one_effcient.pth'))
model_a.eval()

test_feats = []
with torch.no_grad():
    for data in test_loader:
        data = data[0].cuda()
        feat = model_a(data)
        test_feats.append(feat.data.cpu().numpy())
        
test_feats = np.vstack(test_feats)
test_feats = normalize(test_feats)

# 每折保存距离
distance_1 = []
for feat in test_feats[:]:
    dis = np.dot(feat, test_feats.T)
    feat_qe = np.multiply(dis[np.argsort(dis)[::-1][:2]].reshape(2, -1),
            test_feats[np.argsort(dis)[::-1][:2]]).mean(0)
    dis = np.dot(feat_qe, test_feats.T)
    distance_1.append(dis)
distance_1

[array([ 0.99999946,  0.3198834 , -0.01690741, ..., -0.00294943,
        -0.00581286,  0.21934532], dtype=float32),
 array([ 0.31988353,  1.0000002 , -0.01674428, ...,  0.2797472 ,
         0.05742166,  0.21857098], dtype=float32),
 array([-0.01690742, -0.01674427,  1.        , ...,  0.00884337,
        -0.02676789, -0.0822895 ], dtype=float32),
 array([ 0.04320084,  0.10989234,  0.12125073, ...,  0.15630761,
         0.10168619, -0.00022641], dtype=float32),
 array([ 0.09155224,  0.07749283, -0.02091111, ...,  0.01971644,
         0.32124645,  0.05093642], dtype=float32),
 array([ 0.10028163, -0.0417973 ,  0.20578927, ..., -0.08845288,
        -0.03000725,  0.08484295], dtype=float32),
 array([ 0.38881147,  0.3725368 , -0.02028123, ...,  0.12358822,
         0.07594056,  0.18573216], dtype=float32),
 array([ 0.04289604, -0.02232083,  0.13498992, ..., -0.07445642,
        -0.03187206,  0.07393806], dtype=float32),
 array([0.09567646, 0.02363959, 0.0025441 , ..., 0.16003825, 0.06827424,

In [87]:
model_a.load_state_dict(torch.load('two_final.pth'))
model_a.eval()

test_feats = []
with torch.no_grad():
    for data in test_loader:
        data = data[0].cuda()
        feat = model_a(data)
        test_feats.append(feat.data.cpu().numpy())
        
test_feats = np.vstack(test_feats)
test_feats = normalize(test_feats)

# 每折保存距离
distance_2 = []
for feat in test_feats[:]:
    dis = np.dot(feat, test_feats.T)
    feat_qe = np.multiply(dis[np.argsort(dis)[::-1][:2]].reshape(2, -1),
        test_feats[np.argsort(dis)[::-1][:2]]).mean(0)
    dis = np.dot(feat_qe, test_feats.T)
    distance_2.append(dis)
    
distance_2

[array([ 0.9999999 ,  0.22317624, -0.04733918, ...,  0.09620396,
        -0.02749076,  0.27530658], dtype=float32),
 array([ 0.22317618,  0.99999934, -0.09562052, ...,  0.3622707 ,
         0.00852664,  0.10493977], dtype=float32),
 array([-4.7339194e-02, -9.5620573e-02,  1.0000004e+00, ...,
         5.2581869e-02, -1.8166155e-02, -8.3469646e-04], dtype=float32),
 array([0.04054725, 0.07725883, 0.17129856, ..., 0.07638515, 0.07247384,
        0.0214952 ], dtype=float32),
 array([ 0.11168199, -0.06914572, -0.08292134, ..., -0.02435419,
         0.3326865 ,  0.04041754], dtype=float32),
 array([ 0.10303181, -0.04224646,  0.14244382, ..., -0.10820918,
        -0.13040923,  0.00342928], dtype=float32),
 array([ 0.2707744 ,  0.37569225, -0.09052817, ...,  0.21757036,
         0.01388425,  0.29745382], dtype=float32),
 array([-0.0160963 ,  0.01175381,  0.15830153, ..., -0.03266117,
         0.04652368,  0.07966216], dtype=float32),
 array([ 0.12821019, -0.0221277 ,  0.05090334, ...,  0.15443

In [68]:
model_a.load_state_dict(torch.load('three_effcient.pth'))
model_a.eval()

test_feats = []
with torch.no_grad():
    for data in test_loader:
        data = data[0].cuda()
        feat = model_a(data)
        test_feats.append(feat.data.cpu().numpy())
        
test_feats = np.vstack(test_feats)
test_feats = normalize(test_feats)

# 每折保存距离
distance_3 = []
for feat in test_feats[:]:
    dis = np.dot(feat, test_feats.T)
    feat_qe = np.multiply(dis[np.argsort(dis)[::-1][:2]].reshape(2, -1),test_feats[np.argsort(dis)[::-1][:2]]).mean(0)
    dis = np.dot(feat_qe, test_feats.T)
    distance_3.append(dis)
    
distance_3

[array([ 1.0000002 ,  0.29759413, -0.00407454, ...,  0.19275305,
        -0.01836714,  0.1673429 ], dtype=float32),
 array([ 0.29759413,  1.        , -0.03471044, ...,  0.3887102 ,
         0.07059977,  0.13019887], dtype=float32),
 array([-0.00407454, -0.03471044,  0.9999995 , ..., -0.02830489,
        -0.00240749, -0.05730473], dtype=float32),
 array([ 0.02101957,  0.07691745,  0.11760764, ...,  0.06908366,
         0.0925955 , -0.01662869], dtype=float32),
 array([0.18777198, 0.03220204, 0.08236337, ..., 0.03693729, 0.2684585 ,
        0.01089749], dtype=float32),
 array([ 0.12909813, -0.04141798,  0.2800129 , ..., -0.08373556,
         0.0234496 ,  0.01371022], dtype=float32),
 array([ 0.31519842,  0.34465787, -0.09435777, ...,  0.3431446 ,
        -0.01205908,  0.3554704 ], dtype=float32),
 array([-0.05748518, -0.03194991,  0.21046144, ..., -0.04115251,
         0.0764765 ,  0.08628373], dtype=float32),
 array([0.01785327, 0.00958483, 0.00133985, ..., 0.0878396 , 0.04203681,
     

In [88]:
model_b.load_state_dict(torch.load('four_final.pth'))
model_b.eval()

test_feats = []
with torch.no_grad():
    for data in test_loader:
        data = data[0].cuda()
        feat = model_b(data)
        test_feats.append(feat.data.cpu().numpy())
        
test_feats = np.vstack(test_feats)
test_feats = normalize(test_feats)

# 每折保存距离
distance_4 = []
for feat in test_feats[:]:
    dis = np.dot(feat, test_feats.T)
    feat_qe = np.multiply(dis[np.argsort(dis)[::-1][:2]].reshape(2, -1),
            test_feats[np.argsort(dis)[::-1][:2]]).mean(0)
    dis = np.dot(feat_qe, test_feats.T)
    distance_4.append(dis)
    
distance_4

[array([ 0.9999999 ,  0.35403144, -0.02283267, ...,  0.14026059,
         0.06699912,  0.19764619], dtype=float32),
 array([ 0.35403147,  0.9999999 , -0.10576413, ...,  0.32544327,
         0.01822689,  0.28689194], dtype=float32),
 array([-0.02283267, -0.10576415,  1.        , ...,  0.03783609,
         0.0023084 , -0.04281079], dtype=float32),
 array([ 0.04512548,  0.07005288,  0.18989962, ...,  0.09630527,
         0.09148106, -0.0420959 ], dtype=float32),
 array([ 0.11155607, -0.01213593,  0.00901735, ...,  0.02950641,
         0.25573075,  0.03225785], dtype=float32),
 array([ 0.02138187,  0.00566645,  0.17514008, ..., -0.0660039 ,
         0.07145783,  0.09334867], dtype=float32),
 array([ 0.28877044,  0.41784483, -0.07119444, ...,  0.23303781,
        -0.01843779,  0.3302891 ], dtype=float32),
 array([-0.0321448 , -0.00906238,  0.10242864, ..., -0.02786671,
         0.00470562,  0.12073671], dtype=float32),
 array([ 0.09086709,  0.11339452, -0.01671776, ...,  0.1669805 ,
       

In [89]:
model_b.load_state_dict(torch.load('five_final.pth'))
model_b.eval()

test_feats = []
with torch.no_grad():
    for data in test_loader:
        data = data[0].cuda()
        feat = model_b(data)
        test_feats.append(feat.data.cpu().numpy())
        
test_feats = np.vstack(test_feats)
test_feats = normalize(test_feats)

# 每折保存距离
distance_5 = []
for feat in test_feats[:]:
    dis = np.dot(feat, test_feats.T)
    feat_qe = np.multiply(dis[np.argsort(dis)[::-1][:2]].reshape(2, -1),test_feats[np.argsort(dis)[::-1][:2]]).mean(0)
    dis = np.dot(feat_qe, test_feats.T)
    distance_5.append(dis)
    
distance_5

[array([ 1.0000005 ,  0.4896341 , -0.07219046, ...,  0.31204012,
        -0.0772709 ,  0.28978908], dtype=float32),
 array([ 0.48963404,  1.0000004 , -0.03673668, ...,  0.40472254,
        -0.04392053,  0.3287732 ], dtype=float32),
 array([-0.07219046, -0.03673668,  1.0000002 , ...,  0.03946589,
         0.05206836, -0.09968269], dtype=float32),
 array([0.0468124 , 0.12284921, 0.14325938, ..., 0.10720729, 0.01334064,
        0.06232793], dtype=float32),
 array([ 0.12031496, -0.0503806 , -0.02865519, ...,  0.10656196,
         0.2899751 , -0.02094014], dtype=float32),
 array([ 0.01855042, -0.08296483,  0.26388907, ...,  0.00217158,
         0.10527271, -0.02796748], dtype=float32),
 array([ 0.40113345,  0.47279337, -0.1344462 , ...,  0.25541836,
        -0.07792953,  0.3528262 ], dtype=float32),
 array([-0.04861671, -0.03151503,  0.07687061, ...,  0.00114798,
        -0.01784111,  0.0465013 ], dtype=float32),
 array([ 0.07322707,  0.09053733, -0.00403366, ...,  0.07604237,
         0.00

In [102]:
test_distance = []
for i in range(len(distance_5)):
    test_distance.append((0.1 * distance_1[i] + 0.25 * distance_2[i] + 0.15 * distance_3[i] + 0.25 * distance_4[i] + 0.25 * distance_5[i]))
    
test_distance

[array([ 1.        ,  0.3433379 , -0.0378925 , ...,  0.16574419,
        -0.01277699,  0.23772143], dtype=float32),
 array([ 0.3433379 ,  0.9999999 , -0.06641132, ...,  0.3593904 ,
         0.01204038,  0.22153816], dtype=float32),
 array([-0.0378925 , -0.06641134,  1.        , ...,  0.02910957,
         0.00601474, -0.0526567 ], dtype=float32),
 array([0.0405943 , 0.09006708, 0.1558806 , ..., 0.09596774, 0.06838182,
        0.00791486], dtype=float32),
 array([ 0.12320927, -0.02033597, -0.0153764 , ...,  0.03544078,
         0.29199153,  0.01966208], dtype=float32),
 array([ 0.06513391, -0.04027864,  0.2079491 , ..., -0.064416  ,
         0.01209705,  0.02774344], dtype=float32),
 array([ 0.32633048,  0.40553498, -0.09022399, ...,  0.24033715,
        -0.01483557,  0.31703606], dtype=float32),
 array([-0.02854762, -0.01423047,  0.1294684 , ..., -0.0284635 ,
         0.01663131,  0.08206141], dtype=float32),
 array([0.08532173, 0.04925272, 0.00799337, ..., 0.12854506, 0.05267297,
     

In [99]:
# 做了扩展查询
test_submit = []
for i in range(len(test_path)):
    dis = test_distance[i]
    path = test_path[i]
    pred = [x.split('/')[-1] for x in test_path[np.where(dis > 0.59)[0]]]
    
    if len(pred) <= 1:
        ids = dis.argsort()[::-1]
        pred = [x.split('/')[-1] for x in test_path[ids[:2]]]
    print(pred)
    test_submit.append([
        path.split('/')[-1],
        pred
    ])

['000005.jpg', '001040.jpg', '001526.jpg', '012306.jpg', '014146.jpg']
['000011.jpg', '012317.jpg']
['000012.jpg', '012319.jpg']
['000015.jpg', '000418.jpg']
['000017.jpg', '001949.jpg', '012331.jpg', '015780.jpg']
['000021.jpg', '012338.jpg']
['000025.jpg', '012347.jpg']
['000029.jpg', '000136.jpg', '000174.jpg', '000176.jpg', '000273.jpg', '000454.jpg', '000772.jpg', '000890.jpg', '001059.jpg', '001646.jpg', '001665.jpg', '012357.jpg', '012546.jpg', '012616.jpg', '012620.jpg', '012804.jpg', '013118.jpg', '013688.jpg', '013892.jpg', '014181.jpg', '015232.jpg', '015262.jpg']
['000031.jpg', '000422.jpg']
['000032.jpg', '003383.jpg']
['000034.jpg', '001393.jpg', '003391.jpg', '004450.jpg']
['000036.jpg', '012366.jpg']
['000037.jpg', '001014.jpg']
['000041.jpg', '012375.jpg']
['000043.jpg', '000736.jpg', '012381.jpg', '013615.jpg']
['000048.jpg', '001019.jpg', '002101.jpg', '012389.jpg', '014111.jpg', '016094.jpg']
['000053.jpg', '012397.jpg']
['000054.jpg', '012398.jpg']
['000056.jpg', '

['002251.jpg', '002256.jpg', '002449.jpg', '002760.jpg', '002840.jpg', '003112.jpg', '010051.jpg', '010057.jpg', '010453.jpg', '011110.jpg', '011289.jpg', '011801.jpg']
['002257.jpg', '003154.jpg']
['002258.jpg', '010061.jpg']
['002263.jpg', '003096.jpg', '010080.jpg', '011771.jpg']
['002251.jpg', '002264.jpg', '002449.jpg', '002925.jpg', '003112.jpg', '010051.jpg', '010081.jpg', '010453.jpg', '011445.jpg', '011801.jpg']
['002265.jpg', '010082.jpg']
['002268.jpg', '003030.jpg', '010086.jpg', '011640.jpg']
['002269.jpg', '010087.jpg']
['002278.jpg', '010113.jpg']
['002280.jpg', '010121.jpg']
['002282.jpg', '010122.jpg']
['002293.jpg', '002349.jpg', '002417.jpg', '002424.jpg', '003074.jpg', '010146.jpg', '010264.jpg', '010398.jpg', '010406.jpg', '011732.jpg']
['002301.jpg', '010165.jpg']
['002306.jpg', '010181.jpg']
['002307.jpg', '003155.jpg', '010183.jpg', '011910.jpg']
['002311.jpg', '002811.jpg', '003208.jpg', '010194.jpg', '011217.jpg', '012019.jpg']
['002315.jpg', '003152.jpg', '01

['004419.jpg', '004879.jpg', '004885.jpg']
['004229.jpg', '004880.jpg']
['004884.jpg', '004980.jpg']
['003752.jpg', '004378.jpg', '004419.jpg', '004759.jpg', '004879.jpg', '004885.jpg']
['004886.jpg', '003966.jpg']
['004567.jpg', '004888.jpg']
['003422.jpg', '003426.jpg', '003527.jpg', '003751.jpg', '003856.jpg', '003926.jpg', '004075.jpg', '004090.jpg', '004238.jpg', '004338.jpg', '004438.jpg', '004814.jpg', '004832.jpg', '004889.jpg', '004904.jpg', '004917.jpg', '004960.jpg', '005028.jpg']
['001968.jpg', '004895.jpg']
['003750.jpg', '004897.jpg']
['003739.jpg', '003751.jpg', '003926.jpg', '003985.jpg', '004069.jpg', '004090.jpg', '004110.jpg', '004121.jpg', '004338.jpg', '004460.jpg', '004814.jpg', '004889.jpg', '004904.jpg', '004917.jpg', '004951.jpg']
['003495.jpg', '003863.jpg', '004162.jpg', '004905.jpg']
['003370.jpg', '003599.jpg', '003783.jpg', '003793.jpg', '003978.jpg', '004083.jpg', '004275.jpg', '004415.jpg', '004492.jpg', '004514.jpg', '004616.jpg', '004778.jpg', '004907.

['008463.jpg', '007569.jpg']
['007162.jpg', '008464.jpg', '009476.jpg']
['005738.jpg', '006030.jpg', '006182.jpg', '006672.jpg', '008469.jpg', '009209.jpg', '009334.jpg', '009603.jpg']
['005587.jpg', '006001.jpg', '008473.jpg', '008857.jpg']
['006576.jpg', '008475.jpg']
['008478.jpg', '008547.jpg', '008974.jpg']
['005519.jpg', '005969.jpg', '006217.jpg', '006738.jpg', '007567.jpg', '008226.jpg', '008480.jpg', '009819.jpg']
['005252.jpg', '008481.jpg', '009099.jpg']
['006570.jpg', '008491.jpg']
['008495.jpg', '005698.jpg']
['007790.jpg', '008496.jpg', '008827.jpg']
['006004.jpg', '006718.jpg', '008504.jpg', '009087.jpg']
['008509.jpg', '009187.jpg']
['007927.jpg', '008513.jpg']
['007310.jpg', '008517.jpg']
['005234.jpg', '005859.jpg', '005911.jpg', '006031.jpg', '006035.jpg', '006219.jpg', '006231.jpg', '006336.jpg', '006734.jpg', '007393.jpg', '007485.jpg', '007713.jpg', '007956.jpg', '008177.jpg', '008524.jpg', '009081.jpg']
['006040.jpg', '007315.jpg', '007999.jpg', '008525.jpg']
['0

['012459.jpg', '013443.jpg']
['012465.jpg', '012894.jpg', '012947.jpg', '013212.jpg', '014422.jpg', '015317.jpg']
['000085.jpg', '012466.jpg']
['012293.jpg', '012467.jpg', '014434.jpg', '014846.jpg']
['012348.jpg', '012469.jpg']
['000089.jpg', '000280.jpg', '012475.jpg', '012818.jpg']
['000056.jpg', '000091.jpg', '001883.jpg', '001970.jpg', '002051.jpg', '012401.jpg', '012478.jpg', '015642.jpg', '015822.jpg', '015988.jpg']
['000092.jpg', '012479.jpg']
['000093.jpg', '000440.jpg', '000775.jpg', '012480.jpg', '013102.jpg', '013693.jpg']
['004101.jpg', '012487.jpg']
['011558.jpg', '012494.jpg', '013817.jpg', '015554.jpg']
['012496.jpg', '013504.jpg', '014577.jpg']
['000108.jpg', '012504.jpg']
['000109.jpg', '012505.jpg']
['000115.jpg', '012513.jpg']
['000116.jpg', '000301.jpg', '012515.jpg', '012863.jpg']
['000119.jpg', '012517.jpg']
['012520.jpg', '013814.jpg', '014989.jpg', '015037.jpg']
['012524.jpg', '015617.jpg']
['000123.jpg', '012525.jpg']
['012532.jpg', '013191.jpg']
['000127.jpg'

['013346.jpg', '015807.jpg']
['000675.jpg', '001436.jpg', '001960.jpg', '002662.jpg', '003033.jpg', '010880.jpg', '011647.jpg', '013516.jpg', '014842.jpg', '015809.jpg']
['015810.jpg', '015952.jpg']
['001962.jpg', '015811.jpg']
['012799.jpg', '014774.jpg', '015812.jpg']
['015813.jpg', '014038.jpg']
['001965.jpg', '015815.jpg']
['015816.jpg', '012569.jpg']
['001070.jpg', '001304.jpg', '001967.jpg', '014199.jpg', '014608.jpg', '015818.jpg']
['012663.jpg', '015819.jpg', '015885.jpg']
['000091.jpg', '000276.jpg', '000983.jpg', '001970.jpg', '012478.jpg', '012810.jpg', '014038.jpg', '015822.jpg']
['015825.jpg', '014655.jpg']
['015270.jpg', '015829.jpg', '016072.jpg']
['012565.jpg', '013605.jpg', '015831.jpg']
['001973.jpg', '002127.jpg', '015832.jpg', '016148.jpg']
['015835.jpg', '016089.jpg']
['001200.jpg', '001825.jpg', '001976.jpg', '014442.jpg', '015540.jpg', '015838.jpg']
['001977.jpg', '015839.jpg']
['013175.jpg', '013844.jpg', '015840.jpg']
['014549.jpg', '015843.jpg']
['001980.jpg',

In [100]:
test_submit = pd.DataFrame(test_submit, columns=['name', 'label'])
test_submit['label'] = test_submit['label'].apply(lambda x: ' '.join(x))

test_submit

,name,label
0,000005.jpg,000005.jpg 001040.jpg 001526.jpg 012306.jpg 01...
1,000011.jpg,000011.jpg 012317.jpg
2,000012.jpg,000012.jpg 012319.jpg
3,000015.jpg,000015.jpg 000418.jpg
4,000017.jpg,000017.jpg 001949.jpg 012331.jpg 015780.jpg
...,...,...
4815,016148.jpg,001973.jpg 002127.jpg 015832.jpg 016148.jpg
4816,016154.jpg,002131.jpg 016154.jpg
4817,016168.jpg,002117.jpg 002137.jpg 016122.jpg 016168.jpg
4818,016171.jpg,014843.jpg 016171.jpg


In [101]:
test_submit.to_csv('submit818_2.csv',index=None)